# Setup

In [1]:
import os
import glob
import json
import yaml
import requests
import pandas as pd

In [2]:
from tqdm.auto import tqdm
tqdm.pandas()

In [3]:
# a github access token is required for a large number of requests
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

token = cfg['access_token']
headers = {'Authorization': 'token ' + token}

In [4]:
file_dict = {'deaths': 'time_series_covid19_deaths_US.csv',
             'cases': 'time_series_covid19_confirmed_US.csv'}

In [5]:
fips = pd.read_csv('locations.csv')

# Download raw files

In [6]:
for target in file_dict:
    # retrieve information about all commits that modified the file we want
    all_commits = []

    page = 0
    while True:
        page += 1
        r = requests.get('https://api.github.com/repos/CSSEGISandData/COVID-19/commits',
                         params = {'path': f'csse_covid_19_data/csse_covid_19_time_series/{file_dict[target]}',
                                   'page': str(page)},
                         headers = headers)
        
        if (not r.ok) or (r.text == '[]'):
            break
        
        all_commits += json.loads(r.text or r.content)
    
    # dataframe of commit shas and corresponding commit dates
    commits_df = pd.DataFrame({'date_time': [commit['commit']['author']['date'] for commit in all_commits],
                               'sha': [entry['sha'] for entry in all_commits]})

    commits_df.date_time = pd.to_datetime(commits_df.date_time)
    commits_df['date'] = commits_df.date_time.dt.date
    
    # only consider last commit of each day
    commits_df = commits_df.loc[commits_df.groupby('date')['date_time'].idxmax()]
    
    # download and save the csvs
    print(f'Downloading files: {target}')
    for _, row in tqdm(commits_df.iterrows(), total=commits_df.shape[0]):
        result_path =  f'data/raw/{row.date}_JHU_raw_{target}.csv'
        
        # check if file already exists, don't save today's file as it might get updated again
        if not os.path.isfile(result_path) and row.date != pd.Timestamp('today').date():
            df = pd.read_csv(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/{row.sha}' \
                             f'/csse_covid_19_data/csse_covid_19_time_series/{file_dict[target]}')
            df.to_csv(result_path, index=False)

  0%|          | 0/517 [00:00<?, ?it/s]

  0%|          | 0/517 [00:00<?, ?it/s]

# Process files

In [7]:
def process_file(filepath, weekly=True):
    df = pd.read_csv(filepath)
    
    # extract target from filepath
    target = filepath.split('_')[-1][:-4]

    df.drop(columns=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Country_Region', 
                     'Lat', 'Long_', 'Combined_Key', 'Population'], errors='ignore', inplace=True)

    df = df.groupby('Province_State').sum().reset_index()
    df = pd.melt(df, id_vars=['Province_State'])
    df.columns = ['location_name', 'date', 'value']
    df.date = pd.to_datetime(df.date)

    df = df.merge(fips[['location', 'location_name']], how='left')
    df = df[['date', 'location', 'location_name', 'value']].sort_values(['date', 'location'])
    
    if weekly:
        df = df[df.date.dt.day_name() == 'Saturday'].reset_index(drop=True)
    
    df.drop(columns=['location_name'], inplace=True)

    # compute national level
    us = df.groupby('date')['value'].sum().reset_index()
    us['location'] = 'US'
    df.dropna(inplace=True) # drop Diamond Princess and Grand Princess (they are included in the national level)
    df = pd.concat([df, us]).sort_values(['date', 'location']).reset_index(drop=True)

    df.to_csv(f'data/cumulative_{target}/jhu_cumulative_{target}_as_of_{row.date.date()}.csv', index=False)

    # compute incidence
    df.value = df.groupby(['location'])['value'].diff()
    df.dropna(inplace=True)
    df.value = df.value.astype(int)

    df.to_csv(f'data/incident_{target}/jhu_incident_{target}_as_of_{row.date.date()}.csv', index=False)
    
    return df

In [8]:
files = glob.glob('data/raw/*')

# only consider data from Monday
file_df = pd.DataFrame({'filepath': files})
file_df['date'] = file_df.filepath.transform(lambda x: x.split('\\')[1][:10])
file_df.date = pd.to_datetime(file_df.date)
file_df = file_df[file_df.date.dt.day_name() == 'Monday'].reset_index(drop=True)

print('Processing files:')
for _, row in tqdm(file_df.iterrows(), total=file_df.shape[0]):
    temp = process_file(row['filepath'], weekly=True)

Processing files:


  0%|          | 0/146 [00:00<?, ?it/s]